<a href="https://colab.research.google.com/github/PingPingE/Learn_ML_DL/blob/main/Practice/Hands_On_ML/ch13-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 텐서플로에서 데이터 적재와 전처리하기

- 데이터의 양이 많은 경우, 이를 Binary로 serialization한 후 파일 형태로 저장하고 있다가, 이를 다시 읽어들이는 형태로 처리하면 속도 상의 이득을 기대할 수 있다.
- 이를 위해 protocol buffer형태로 serialization을 수행해서 저장할 수 있는 TFRecord 포맷이 필요한 것

*참고: [링크](http://solarisailab.com/archives/2603)<br>
*protocol buffer: 구글이 개발한 이식성과 확장성이 좋고 효율적인 이진 포맷

# TFRecord 포맷
- <strong>크기가 다른 연속된 이진 레코드</strong>를 저장하는 단순한 이진 포맷
- 각 레코드는 길이, 길이가 올바른지 체크하는 CRC 체크섬, 실제 데이터, 데이터를 위한 CRC 체크섬으로 구성
- <strong>tf.io.TFRecordWriter 클래스</strong>를 사용해서 TFRecord를 손쉽게 만든다.

In [2]:
import tensorflow as tf
with tf.io.TFRecordWriter("my_data.tfrecord") as f: #TFRecord 만들기
  f.write(b"This is the first record")
  f.write(b"And this is the second record")

- tf.data.TFRecordDataset을 사용해서 하나 이상의 TFRecord를 읽을 수 있다.

In [ ]:
filepaths=['my_data.tfrecord']
dataset=tf.data.TFRecordDataset(filepaths) 
for item in dataset:
  print(item)

tf.Tensor(b'This is the first record', shape=(), dtype=string)
tf.Tensor(b'And this is the second record', shape=(), dtype=string)


------------
기본적으로 TFRecordDataset은 파일을 하나씩 차례로 읽는다.
<br>-> num_parallel_reads를 지정해서 여러 파일에서 레코드를 번갈아 읽을 수 있다(DataAPI의 list_files()+interleave()와 동일한 결과)

## 압축된 TFRecord 파일
- 네트워크를 통해 읽어야 하는 경우, TFRecord파일을 압축할 필요가 있다.
- options 매개변수를 이용해서 압축된 TFRecord 파일을 만들 수 있다.

In [ ]:
options=tf.io.TFRecordOptions(compression_type="GZIP") #압축 형식 지정
with tf.io.TFRecordWriter("my_compressed.tfrecord", options) as f:
  f.write(b"(compressed)This is the first record")
  f.write(b"(compressed)And this is the second record")

In [ ]:
filepaths=['my_compressed.tfrecord']
dataset=tf.data.TFRecordDataset(filepaths, compression_type="GZIP") #압축된걸 읽을 때도 compression_type을 지정해야 한다
for item in dataset:
  print(item)

tf.Tensor(b'(compressed)This is the first record', shape=(), dtype=string)
tf.Tensor(b'(compressed)And this is the second record', shape=(), dtype=string)


## 프로토콜 버퍼 개요
- 각 레코드는 어떤 이진 포맷도 사용할 수 있지만, 일반적으로 TFRecord는 직렬화된 프로토콜 버퍼를 담는다.


- 프로토콜 버퍼의 정의
```
syntax="proto3"#포맷 버전 3
#Person 객체 생성
message Person{
    string name=1;
    int32 id=2;
    repeated string email=3; #string 타입의 email필드를 하나 이상 가진다.
}
```

------
- 프로토콜 버퍼 객체는 직렬화해서 전송된 것을 의미하므로 message라고 부른다.
- 각 필드에 부여된 1,2,3 값은 필드 식별자로, 레코드의 이진 표현에 사용된다.

### 텐서플로 프로토콜 버퍼
- TFRecord 파일에서 사용하는 전형적인 주요 프로토콜 버퍼는 <strong>데이터셋에 있는 하나의 샘플을 표현하는 Example 프로토콜 버퍼다.</strong>



```
syntax="proto3"
message BytesList {repeated bytes value=1;}
message FloatList {repeated float value=1 [packed=True];}
message Int64List {repeated int64 value=1 [packed=True];}
message Feature{
    oneof kind{
        BytesList bytes_list=1;
        FloatList float_list=2;
        Int64List int64_list=3;
    }
};
message Features {map<string, Feature> feature=1; };
message Example {Features features=1; };
```



--------
- [packed=True]는 효율적인 인코딩을 위해 반복적인 수치 필드에 사용된다.
- Feature는 BytesList, FloatList, Int64List 중 하나를 담는다.
- Features는 특성이름과 특성값을 매핑한 딕셔너리
- Example은 하나의 Features 객체를 가진다.

In [ ]:
from tensorflow.train import BytesList, FloatList, Int64List
from tensorflow.train import Feature, Features, Example

person_example=Example(
    features=Features(
        feature={
            "name": Feature(bytes_list=BytesList(value=[b"Alice"])),
            "id":Feature(int64_list=Int64List(value=[123])),
            "emails":Feature(bytes_list=BytesList(value=[b"a@b.com", b"c@d.com"]))
        }
    )
)

In [ ]:
with tf.io.TFRecordWriter("my_contacts.tfrecord") as f:
  f.write(person_example.SerializeToString()) #직렬화하고 결과 데이터를 tfrecord파일에 저장

-----------------------
보통은 현재 포맷(csv파일 같은)을 읽어 각 샘플마다 하나의 Example 프로토콜 버퍼를 생성하고, 위와 같이 저장한다.

### Example 프로토콜 버퍼를 읽고 파싱하기 

In [ ]:
#설명 딕셔너리
feature_description={
    "name":tf.io.FixedLenFeature([], tf.string,default_value=""),
    "id":tf.io.FixedLenFeature([], tf.int64, default_value=0),
    "emails":tf.io.VarLenFeature(tf.string),  #repeated이므로 특성 리스트 길이가 가변적
}

In [ ]:
for serialized_example in tf.data.TFRecordDataset(["my_contacts.tfrecord"]):
  parsed_example=tf.io.parse_single_example(serialized_example, feature_description) #parse_example()로 배치 단위로 파싱할 수도 있다.

In [ ]:
for serialized_example in tf.data.TFRecordDataset(["my_contacts.tfrecord"]):
  parsed_example=tf.io.parse_single_example(serialized_example) #설명 딕셔너리 없으면 error

TypeError: ignored

- 가변 길이 특성은 희소 텐서로 파싱된다(고정 길이 특성은 보통의 텐서)

In [ ]:
tf.sparse.to_dense(parsed_example['emails'], default_value=b"") #희소 -> 밀집 변환

<tf.Tensor: shape=(2,), dtype=string, numpy=array([b'a@b.com', b'c@d.com'], dtype=object)>

In [ ]:
parsed_example['emails'].values #근데 굳이 변환 할 필요 X 바로 참조하는 것이 더 간단

<tf.Tensor: shape=(2,), dtype=string, numpy=array([b'a@b.com', b'c@d.com'], dtype=object)>

### SequenceExample 프로토콜 버퍼를 사용해서 리스트의 리스트 다루기

- SequenceExample 프로토콜 버퍼의 정의

```
message FeatureList { repeated Feature feature=1;};
message FeatureLists {map<string, FeatureList> feature_list =1;};
message SequenceExample{
  Features context=1;
  FeatureLists feature_lists=2;
};
```



------------
- 문맥 데이터를 위한 하나의 Features 객체와 이름이 있는 한 개 이상의 FeatureList를 가진 FeatureLists 객체를 포함한다.
- Feature 객체는 바이트 스트링, 64비트 정수, 실수의 리스트 중 하나
- 나머지 과정은 Example을 만들고 직렬화하고 파싱하는 것과 비슷

# 입력 특성 전처리
- 신경망을 위해 데이터를 준비하려면 일반적으로 모든 특성을 <strong>수치  특성</strong>으로 변환하고 <strong>정규화</strong>해야 한다.
- 방법: pandas, numpy, scikit learn, Data API, 전처리 층(Lambda 층 등)포함 등

## 전처리 담당 Lambda 층 포함하기

- 방법 1

In [5]:
from tensorflow import keras
import numpy as np
X_train=tf.constant([1.,2.,3.])
means=np.mean(X_train, axis=0, keepdims=True)
stds=np.std(X_train, axis=0, keepdims=True)
eps=keras.backend.epsilon()

model=keras.models.Sequential([
                              keras.layers.Lambda(lambda inputs: (inputs-means)/(stds+eps)),
                              keras.layers.Dense(5),
                              keras.layers.Dense(1)
])

- 방법 2

In [20]:
class Standardization(keras.layers.Layer):
  def adapt(self, data_sample):
    self.means_ = np.mean(data_sample, axis=0, keepdims=True)
    self.stds_ = np.std(data_sample, axis=0, keepdims=True)
  def call(self, inputs):
    return (inputs-self.means_)/(self.stds_+keras.backend.epsilon())

std_layer=Standardization()
std_layer.adapt(X_train)

------------
방법 1은 mean, std를 전역으로 선언해서 Lambda 층에 활용했고, <br>
방법 2는 사용자 정의 층을 만들어서 mean, std를 따로 계산하는 adapt 메서드를 만들었다. 

=> keras에는 "LayerNomalization"과 "BatchNormalization" 층이 구현되어 있다. 
[링크](https://keras.io/api/layers/normalization_layers/)

## 원-핫 벡터를 사용해서 범주형 특성 인코딩하기

## 임베딩을 사용해서 범주형 특성 인코딩하기 

## keras 전처리 층